# VacationPy

In [1]:
# Import dependencies 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import time 
import json
from scipy.stats import linregress 

# Import API key
from config import gkey

In [2]:
# Install jupyter-gmaps with conda
#conda install -c conda-forge gmaps

In [3]:
# Import gmaps 
import gmaps

# Access maps with unique API key
gmaps.configure(api_key = gkey)

## Store Part I results into DataFrame
- Load the csv exported in Part I to a DataFrame

In [4]:
# Import cities data
cities_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
cities_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vaini,-21.2000,-175.2000,73.40,100,20,3.44,TO,1614619855
1,Saint George,37.1041,-113.5841,46.00,20,1,3.18,US,1614619852
2,Gladstone,-23.8500,151.2500,77.00,83,75,5.75,AU,1614619573
3,Kupang,-10.1667,123.5833,76.89,86,100,2.24,ID,1614619857
4,Imbituba,-28.2400,-48.6703,73.40,94,75,3.44,BR,1614619857
...,...,...,...,...,...,...,...,...,...
580,Oyem,1.5995,11.5793,71.40,94,64,4.07,GA,1614620176
581,Vangaindrano,-23.3500,47.6000,76.98,88,99,9.10,MG,1614620176
582,Qūchān,37.1060,58.5095,43.48,57,95,2.01,IR,1614620177
583,Nara,34.6851,135.8049,57.00,69,90,4.05,JP,1614620178


In [5]:
# Check datatypes
cities_df.dtypes

City           object
Latitude      float64
Longitude     float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

## Humidity Heatmap
- Configure gmaps
- Use the Lat and Lng as locations and Humidity as the weight
- Add Heatmap layer to map

In [6]:
# Store latitude and longitude in locations
locations = cities_df[['Latitude', 'Longitude']]
locations

,Latitude,Longitude
0,-21.2000,-175.2000
1,37.1041,-113.5841
2,-23.8500,151.2500
3,-10.1667,123.5833
4,-28.2400,-48.6703
...,...,...
580,1.5995,11.5793
581,-23.3500,47.6000
582,37.1060,58.5095
583,34.6851,135.8049


In [7]:
# Select the humidtiy values
humidity = cities_df['Humidity']

In [8]:
# Plot Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating = False, max_intensity=humidity.max(),
                                 point_radius=3
                                )
# Add Layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria
- Narrow down the cities to fit weather conditions.
- Drop any rows with null values.

In [9]:
# Narrow down the dataframe to find your ideal weather condition
ideal_places = cities_df[(cities_df['Max Temp'] < 80) & 
                         (cities_df['Max Temp'] >70) & 
                         (cities_df['Wind Speed'] < 10) &
                         (cities_df['Cloudiness'] == 0) &
                         (cities_df['Humidity'] < 65)
                        ]
# Drop Nan Values
ideal_places = ideal_places.dropna()
ideal_places

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
156,Salalah,17.0151,54.0924,78.80,61,0,2.30,OM,1614619967
217,Keti Bandar,24.1447,67.4497,71.24,54,0,5.77,PK,1614620001
334,Dwārka,22.2394,68.9678,73.56,63,0,9.46,IN,1614619932
438,Balod,20.7300,81.2047,75.60,22,0,4.18,IN,1614620114
454,Mārkāpur,15.7333,79.2833,75.72,56,0,3.06,IN,1614620121
464,Bamora,24.0552,78.0909,70.97,22,0,6.78,IN,1614620124
570,Adrar,20.5022,-10.0711,76.55,17,0,7.85,MR,1614620171


In [10]:
# Reset Index
ideal_places = ideal_places.reset_index()
ideal_places = ideal_places.drop(columns=['index'])
ideal_places

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Salalah,17.0151,54.0924,78.80,61,0,2.30,OM,1614619967
1,Keti Bandar,24.1447,67.4497,71.24,54,0,5.77,PK,1614620001
2,Dwārka,22.2394,68.9678,73.56,63,0,9.46,IN,1614619932
3,Balod,20.7300,81.2047,75.60,22,0,4.18,IN,1614620114
4,Mārkāpur,15.7333,79.2833,75.72,56,0,3.06,IN,1614620121
5,Bamora,24.0552,78.0909,70.97,22,0,6.78,IN,1614620124
6,Adrar,20.5022,-10.0711,76.55,17,0,7.85,MR,1614620171


## Hotel Map 
- Store into variable names hotel_df
- Add a 'Hotel Name' column to the DataFrame
- Set parameters to search for hotels with 5000 meteres.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
- Plot makers on top of the heatmap.

In [11]:
# Find the first hotel for each city located within 5000mts 
# of the coordinates
# params dictionary to update each iteration
params = {
    "radius":5000,
    "types": 'hotel',
    "keyword":'hotel',
    "key":gkey
}

# Use the Latitude/Longitude we recovered to identify hotels
for index, row in ideal_places.iterrows():
    # get latitude and longitude from df
    lat = row['Latitude']
    lng = row['Longitude']
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # use the search term: 'Hotel' and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # print 
    name_address = name_address.json()
    
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # use try-except to skip any that are missing a data point
    try:
        ideal_places.loc[index, 'Hotel Name'] = name_address['results'][0]['name']
    
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')   
        


Missing field/result... skipping.
Missing field/result... skipping.


In [12]:
ideal_places = ideal_places.dropna(how='any')
ideal_places

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Salalah,17.0151,54.0924,78.80,61,0,2.30,OM,1614619967,Alpha Suites Hotel
2,Dwārka,22.2394,68.9678,73.56,63,0,9.46,IN,1614619932,Lemon Tree Premier
3,Balod,20.7300,81.2047,75.60,22,0,4.18,IN,1614620114,Arvindo Hotel
4,Mārkāpur,15.7333,79.2833,75.72,56,0,3.06,IN,1614620121,Hotel Seven Hills
5,Bamora,24.0552,78.0909,70.97,22,0,6.78,IN,1614620124,OYO 74563 Panchwati


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_places.iterrows()]
locations = ideal_places[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))